In [ ]:
!pip install cherche

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached cherche-1.0.1-py3-none-any.whl


In [ ]:
from cherche import retrieve

In [ ]:

import pandas as pd

from nltk.tokenize import word_tokenize
from string import punctuation
import nltk
from unicodedata import normalize
from nltk.stem import RSLPStemmer
from nltk.util import ngrams
from nltk.tokenize import RegexpTokenizer

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('rslp')



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package rslp to /root/nltk_data...
[nltk_data]   Unzipping stemmers/rslp.zip.


True

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
caminho_query = "/content/gdrive/MyDrive/Colab Notebooks/dados-conle-anonimizado-assunto-notnull (1).csv"

Mounted at /content/gdrive


# Base de dados câmara dos deputados

In [ ]:
df = pd.read_csv("/content/gdrive/MyDrive/Colab Notebooks/proposicao-tema-completo.csv")

In [ ]:
df_assunto= pd.read_csv(caminho_query, encoding='utf-8', delimiter=";")

In [ ]:
arr_assunto = df_assunto.to_numpy()
y,X = arr_assunto[:,0],arr_assunto[:,1]
y = [i.strip() for i in y]

In [ ]:
def verificar(y,top_n):
    for d in top_n:
      if y == df.loc[d["id"],"txtNome"]:
        return 1
    return 0

In [ ]:
def avaliacaoRecall(isPreprocess):

    quant_encontrado=0
    quant_relevante =0
    for l,x in zip(y,X):

        tokenized_query3 = x                   
        if isPreprocess:
            tokenized_query3 = preprocess(x)
    
        top_n = retriever(tokenized_query3)
        #top_n = [df.loc[d["id"],"name"] for d in top_n_stem_l]

        
        quant_relevante+=1
        quant_encontrado+=verificar(l,top_n)
    
    recall = quant_encontrado / quant_relevante
    print("R@20: "+str(recall))


In [ ]:
def tokenizer1(doc):
    return doc.split("\n")

# Word n-gram + pré processamento básico + RSLP

## 18- Letra mínuscula + remoção de pontuação, acentuação e stopword + stemming (RSLP) + bigram

In [ ]:
def _remove_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

def preprocess(txt):
    txt = _remove_acentos(txt)
    stopwords = list(punctuation)

    stemmer = RSLPStemmer()
    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt.lower())
    terms = [stemmer.stem(word) for word in terms if word not in stopwords]
    
    ngram = []
    ngram_2 = list(ngrams(terms, 2))
        
    for w in ngram_2:
        string = w[0] + " " + w[1]
        ngram.append(string)
    
    return "\n".join(ngram)

In [ ]:
documentos = [
    {
        "id":i,
        "name":str(df.loc[i,"txtNome"]).strip(),
        "text":preprocess(str(df.loc[i,"imgArquivoTeorPDF"]))
    }

   for i in range(len(df))
    ]




In [ ]:
retriever = retrieve.BM25Okapi(key="id", on=["text"],tokenizer=tokenizer1, documents=documentos,k=20,k1=1.5,b=0.75,epsilon=0.25)

Recall

In [ ]:
avaliacaoRecall(True)

R@20: 0.5254237288135594


In [ ]:
documentos = None
retriever =None

## 19- Letra mínuscula + remoção de pontuação, acentuação e stopword + stemming (RSLP) + trigram

In [ ]:
def _remove_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

def preprocess(txt):
    txt = _remove_acentos(txt)
    stopwords = list(punctuation)

    stemmer = RSLPStemmer()
    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt.lower())
    terms = [stemmer.stem(word) for word in terms if word not in stopwords]
    
    ngram = []
    ngram_3 = list(ngrams(terms, 3))
        
    for w in ngram_3:
        string = w[0] + " " + w[1] + " " + w[2]
        ngram.append(string)
    
    return "\n".join(ngram)

In [ ]:
documentos = [
    {
        "id":i,
        "name":str(df.loc[i,"txtNome"]).strip(),
        "text":preprocess(str(df.loc[i,"imgArquivoTeorPDF"]))
    }

   for i in range(len(df))
    ]





In [ ]:
retriever = retrieve.BM25Okapi(key="id", on=["text"],tokenizer=tokenizer1, documents=documentos,k=20,k1=1.5,b=0.75,epsilon=0.25)

Recall

In [ ]:
avaliacaoRecall(True)

R@20: 0.4


In [ ]:
documentos = None
retriever =None

## 20- Letra mínuscula + remoção de pontuação, acentuação e stopword + stemming (RSLP) + unigram + bigram

In [ ]:
def _remove_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

def preprocess(txt):
    txt = _remove_acentos(txt)
    stopwords = list(punctuation)

    stemmer = RSLPStemmer()
    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt.lower())
    terms = [stemmer.stem(word) for word in terms if word not in stopwords]
    
    ngram = []
    ngram_1 = list(ngrams(terms, 1))
    ngram_2 = list(ngrams(terms, 2))
    for w in ngram_1:
        ngram.append(w[0])
        
    for w in ngram_2:
        string = w[0] + " " + w[1]
        ngram.append(string)
    
    return "\n".join(ngram)


In [ ]:
documentos = [
    {
        "id":i,
        "name":str(df.loc[i,"txtNome"]).strip(),
        "text":preprocess(str(df.loc[i,"imgArquivoTeorPDF"]))
    }

   for i in range(len(df))
    ]





In [ ]:
retriever = retrieve.BM25Okapi(key="id", on=["text"],tokenizer=tokenizer1, documents=documentos,k=20,k1=1.5,b=0.75,epsilon=0.25)

Recall

In [ ]:
avaliacaoRecall(True)

In [ ]:
documentos = None
retriever =None

# Word n-gram + pré processamento básico + RSLPS

In [ ]:
class RSLP_S:
    def __plural_reduction(self, word):
        excep = ["lápis","cais","mais","crúcis","biquínis","pois","depois","dois","leis" ]
        excep_s = ["aliás","pires","lápis","cais","mais","mas","menos", "férias","fezes","pêsames","crúcis","gás", "atrás","moisés","através","convés","ês", "país","após","ambas","ambos","messias"]

        len_word = len(word)
        new_word = list(word)

        if len_word >= 3:
            if new_word[-1] == 's' and new_word[-2] == 'n':
                new_word[-2] = 'm'
                sing = "".join(new_word)
                sing = sing[:-1]
                return sing

            if new_word[-1] == 's' and new_word[-2] == 'e' and new_word[-3] == 'õ':
                new_word[-3] = 'ã'
                new_word[-2] = 'o'
                sing = "".join(new_word)
                sing = sing[:-1]
                return  sing

            if new_word[-1] == 's' and new_word[-2] == 'e' and new_word[-3] == 'ã':
                if word == 'mães':
                    word = word[:-1]
                    return word
                else:
                    new_word[-2] = 'o'
                    sing = "".join(new_word)
                    sing = sing[:-1]
                    return sing

            if new_word[-1] == 's' and new_word[-2] == 'i' and new_word[-3] == 'a':
                if word != 'cais' and word != 'mais':
                    new_word[-2] = 'l'
                    sing = "".join(new_word)
                    sing = sing[:-1]
                    return sing

            if new_word[-1] == 's' and new_word[-2] == 'i' and new_word[-3] == 'é':
                new_word[-3] = 'e'
                new_word[-2] = 'l'
                sing = "".join(new_word)
                sing = sing[:-1]
                return sing

            if new_word[-1] == 's' and new_word[-2] == 'i' and new_word[-3] == 'e':
                new_word[-3] = 'e'
                new_word[-2] = 'l'
                sing = "".join(new_word)
                sing = sing[:-1]
                return sing

            if new_word[-1] == 's' and new_word[-2] == 'i' and new_word[-3] == 'ó':
                new_word[-3] = 'o'
                new_word[-2] = 'l'
                sing = "".join(new_word)
                sing = sing[:-1]
                return sing

            if new_word[-1] == 's' and new_word[-2] == 'i':
                if word not in excep:
                    new_word[-1] = 'l'
                    sing = "".join(new_word)
                    return sing

            if new_word[-1] == 's' and new_word[-2] == 'e' and new_word[-3] == 'l':
                word = word[:-2]
                return word

            if new_word[-1] == 's' and new_word[-2] == 'e' and new_word[-3] == 'r':
                word = word[:-2]
                return word

            if new_word[-1] == 's':
                if word not in excep_s:
                    word = word[:-1]

        return word

    def stem(self, word):
        word = self.__plural_reduction(word)

        return word

## 18- Letra mínuscula + remoção de pontuação, acentuação e stopword + stemming (RSLPS) + bigram

In [ ]:
def _remove_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

def preprocess(txt):
    txt = _remove_acentos(txt)
    stopwords = list(punctuation)

    stemmer = RSLP_S()
    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt.lower())
    terms = [stemmer.stem(word) for word in terms if word not in stopwords]
    
    ngram = []
    ngram_2 = list(ngrams(terms, 2))
        
    for w in ngram_2:
        string = w[0] + " " + w[1]
        ngram.append(string)
    
    return "\n".join(ngram)

In [ ]:
documentos = [
    {
        "id":i,
        "name":str(df.loc[i,"txtNome"]).strip(),
        "text":preprocess(str(df.loc[i,"imgArquivoTeorPDF"]))
    }

   for i in range(len(df))
    ]




In [ ]:
retriever = retrieve.BM25Okapi(key="id", on=["text"],tokenizer=tokenizer1, documents=documentos,k=20,k1=1.5,b=0.75,epsilon=0.25)

Recall

In [ ]:
avaliacaoRecall(True)

In [ ]:
documentos = None
retriever =None

## 19- Letra mínuscula + remoção de pontuação, acentuação e stopword + stemming (RSLPS) + trigram

In [42]:
def _remove_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

def preprocess(txt):
    txt = _remove_acentos(txt)
    stopwords = list(punctuation)

    stemmer = RSLP_S()
    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt.lower())
    terms = [stemmer.stem(word) for word in terms if word not in stopwords]
    
    ngram = []
    ngram_3 = list(ngrams(terms, 3))
        
    for w in ngram_3:
        string = w[0] + " " + w[1] + " " + w[2]
        ngram.append(string)
    
    return "\n".join(ngram)

In [43]:
documentos = [
    {
        "id":i,
        "name":str(df.loc[i,"txtNome"]).strip(),
        "text":preprocess(str(df.loc[i,"imgArquivoTeorPDF"]))
    }

   for i in range(len(df))
    ]





In [44]:
retriever = retrieve.BM25Okapi(key="id", on=["text"],tokenizer=tokenizer1, documents=documentos,k=20,k1=1.5,b=0.75,epsilon=0.25)

Recall

In [45]:
avaliacaoRecall(True)

R@20: 0.3864406779661017


In [ ]:
documentos = None
retriever =None

## 20- Letra mínuscula + remoção de pontuação, acentuação e stopword + stemming (RSLPS) + unigram + bigram

In [ ]:
def _remove_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

def preprocess(txt):
    txt = _remove_acentos(txt)
    stopwords = list(punctuation)

    stemmer = RSLP_S()
    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt.lower())
    terms = [stemmer.stem(word) for word in terms if word not in stopwords]
    
    ngram = []
    ngram_1 = list(ngrams(terms, 1))
    ngram_2 = list(ngrams(terms, 2))
    for w in ngram_1:
        ngram.append(w[0])
        
    for w in ngram_2:
        string = w[0] + " " + w[1]
        ngram.append(string)
    
    return "\n".join(ngram)


In [ ]:
documentos = [
    {
        "id":i,
        "name":str(df.loc[i,"txtNome"]).strip(),
        "text":preprocess(str(df.loc[i,"imgArquivoTeorPDF"]))
    }

   for i in range(len(df))
    ]





In [ ]:
retriever = retrieve.BM25Okapi(key="id", on=["text"],tokenizer=tokenizer1, documents=documentos,k=20,k1=1.5,b=0.75,epsilon=0.25)

Recall

In [ ]:
avaliacaoRecall(True)

R@20: 0.5559322033898305


In [ ]:
documentos = None
retriever =None